In [1]:
import torch 
import pandas as pd
from torch.utils.data import DataLoader

from segmentation.config import CFG
from segmentation.models.unet import unet
from segmentation.scr.utils import losses, transforms
from segmentation.scr.utils.utils import set_seed
from segmentation.scr.tilling_dataset import Tilling_Dataset
from segmentation.scr.train_function import train_model

from colorama import Fore, Style
c_  = Fore.GREEN
sr_ = Style.RESET_ALL

#from segmentation.scr
pd.options.mode.chained_assignment = None

C:\Users\123\AppData\Local\Temp\ipykernel_19396\1548443133.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
c:\Users\123\Desktop\human vasculature\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
loss_fn = losses.DiceLoss(smooth=1)
x = torch.zeros((4,1,256,256))
y = torch.zeros((4,256,256))
loss_fn(x,y)

tensor(0.)

In [14]:
loss_fn = DiceLoss()
x = torch.zeros((4,1,256,256))
y = torch.zeros((4,256,256))
loss_fn(x,y)

tensor(1.0000)

In [2]:
train_transform  = transforms.get_transform(transform_type='train')
val_transform = transforms.get_transform(transform_type='val')

In [3]:
train_dataset = Tilling_Dataset(
    name_data='kidney_1_tilling',
    path_to_df=CFG.path_df_kidney_1_till,
    use_random_sub=True,
    empty_tile_pct=0,
    sample_limit=3000,
    random_seed=CFG.random_seed,
    transform=train_transform 
)


Dataset contains 1528 empty and 12146 non-empty tiles.
Sample 0 empty and 3000 non-empty tiles.


In [4]:
val_dataset = Tilling_Dataset(
    name_data='kidney_3_tilling',
    path_to_df=CFG.path_df_kidney_3_till,
    use_random_sub=True,
    empty_tile_pct=6,
    sample_limit=1000,
    random_seed=CFG.random_seed,
    transform=val_transform
    )

Dataset contains 2847 empty and 3165 non-empty tiles.
Sample 60 empty and 940 non-empty tiles.


In [5]:
set_seed(CFG.random_seed)

train_loader = DataLoader(train_dataset, batch_size=CFG.train_batch_size, num_workers=2, shuffle=True, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=CFG.valid_batch_size, num_workers=2, shuffle=False, pin_memory=True)

In [6]:
model = unet.UNet(n_channels=3, n_classes=1, bilinear=True).to(CFG.device)
num_epoch = 60
loss_fn = losses.BCE_DICE(mode="SUM")
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4) 
sheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer=optimizer, mode='max', patience=4, factor=0.6, verbose=True, threshold=1e-3)
device = CFG.device

In [7]:
train_model(model=model, 
                optimizer=optimizer, 
                loss_func=loss_fn, 
                train_loader=train_loader, 
                val_loader=val_loader, 
                num_epochs=num_epoch, 
                scheduler=sheduler, 
                device =CFG.device,
                path_to_save=CFG.path_to_save_state_model)


Epoch 1/60
----------


Train :  24%|██▍       | 182/750 [46:16<2:24:44, 15.29s/it, epoch=182, gpu_mem=11.53 GB, lr=0.00030, train_loss=0.0858]